In [52]:
pip install google-cloud

Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install --ignore-installed google-cloud-storage

  Using cached google_cloud_storage-1.28.1-py2.py3-none-any.whl (80 kB)
  Using cached google_cloud_core-1.3.0-py2.py3-none-any.whl (26 kB)
  Using cached google_auth-1.14.2-py2.py3-none-any.whl (89 kB)
  Using cached google_resumable_media-0.5.0-py2.py3-none-any.whl (38 kB)
  Using cached google_api_core-1.17.0-py2.py3-none-any.whl (70 kB)
  Using cached six-1.14.0-py2.py3-none-any.whl (10 kB)
  Using cached cachetools-4.1.0-py3-none-any.whl (10 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.0-py2.py3-none-any.whl (38 kB)
  Using cached setuptools-46.1.3-py3-none-any.whl (582 kB)
Processing ./.cache/pip/wheels/4c/a1/71/5e427276ceeff277fd76878d1b19fbf4587a2845015d86864b/googleapis_common_protos-1.51.0-py3-none-any.whl
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached protobuf-3.11.3-cp37-cp37m-manylinux1_x86_64.whl (1.3 MB)
  Using cached pyasn1-0.4.8-py2.py3-no

In [54]:
from google.cloud import storage
import os
from io import BytesIO
import pandas as pd

#Connect to GCP bucket and assign the bucket_name and specify the file name
bucket_name = "jzhurecommendation1" #Assign the bucket name where your file is stored
storage_client = storage.Client()

bucket = storage_client.get_bucket(bucket_name)

In [56]:
blob0 = storage.blob.Blob("user_product_rating0.csv",bucket)
blob1 = storage.blob.Blob("user_product_rating1.csv",bucket)
blob2 = storage.blob.Blob("user_product_rating2.csv",bucket)
blob3 = storage.blob.Blob("user_product_rating3.csv",bucket)
blob4 = storage.blob.Blob("user_product_rating4.csv",bucket)
blob5 = storage.blob.Blob("user_product_rating5.csv",bucket)
blob6 = storage.blob.Blob("user_product_rating6.csv",bucket)
blob7 = storage.blob.Blob("user_product_rating7.csv",bucket)
blob8 = storage.blob.Blob("user_product_rating8.csv",bucket)
blob9 = storage.blob.Blob("user_product_rating9.csv",bucket)
blob10 = storage.blob.Blob("user_product_rating10.csv",bucket)
content0 = blob0.download_as_string()
train0 = pd.read_csv(BytesIO(content0))
content1 = blob1.download_as_string()
train1 = pd.read_csv(BytesIO(content1))
content2 = blob2.download_as_string()
train2 = pd.read_csv(BytesIO(content2))
content3 = blob3.download_as_string()
train3 = pd.read_csv(BytesIO(content3))
content4 = blob4.download_as_string()
train4 = pd.read_csv(BytesIO(content4))
content5 = blob5.download_as_string()
train5 = pd.read_csv(BytesIO(content5))
content6 = blob6.download_as_string()
train6 = pd.read_csv(BytesIO(content6))
content7 = blob7.download_as_string()
train7 = pd.read_csv(BytesIO(content7))
content8 = blob8.download_as_string()
train8 = pd.read_csv(BytesIO(content8))
content9 = blob9.download_as_string()
train9 = pd.read_csv(BytesIO(content9))
content10 = blob10.download_as_string()
train10 = pd.read_csv(BytesIO(content10))
df_list = [train0,train1,train2,train3,train4,train5,train6,train7,train8,train9,train10]
df = pd.concat(df_list)

In [58]:
# impport everything we need for the rec sys
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession

In [59]:
# initialize spark session
spark = SparkSession.builder.appName('Recommendation_system').getOrCreate()

In [60]:
df = spark.createDataFrame(df)
df.show()

+----------+-------+----------+----+--------+----------------+----+------+
|Unnamed: 0|user_id|product_id|cart|purchase|remove_from_cart|view|rating|
+----------+-------+----------+----+--------+----------------+----+------+
|         0| 465496|   5769989| 0.0|     0.0|             0.0| 1.0|   1.0|
|         1| 465496|   5865524| 0.0|     0.0|             0.0| 1.0|   1.0|
|         2| 465496|   5865526| 0.0|     0.0|             0.0| 1.0|   1.0|
|         3|1120748|      5240| 0.0|     0.0|             0.0| 1.0|   1.0|
|         4|1180452|   5881337| 0.0|     0.0|             0.0| 1.0|   1.0|
|         5|1458813|   5542414| 0.0|     0.0|             0.0| 1.0|   1.0|
|         6|1458813|   5711078| 0.0|     0.0|             0.0| 1.0|   1.0|
|         7|1458813|   5711087| 0.0|     0.0|             0.0| 2.0|   2.0|
|         8|1458813|   5711117| 0.0|     0.0|             0.0| 1.0|   1.0|
|         9|1458813|   5711122| 0.0|     0.0|             0.0| 1.0|   1.0|
|        10|1458813|   57

In [40]:
# create testing and training data
(training, test) = df.randomSplit([0.8, 0.2])

In [41]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="product_id", ratingCol="rating",coldStartStrategy="drop")

In [42]:
model = als.fit(training)

In [43]:
# generate predictions and evaluations
predictions = model.transform(test)

In [44]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

In [45]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 5.998085713344712


In [46]:
#recommendations
userRecs = model.recommendForAllUsers(10) # top 10 product recommendations for each user
ProductRecs = model.recommendForAllItems(10) # top 10 user recommendations for each product

In [47]:
userRecs.show()

+---------+--------------------+
|  user_id|     recommendations|
+---------+--------------------+
| 40475278|[[5688132, 21.537...|
| 43713532|[[5585632, 65.234...|
| 44772115|[[5864578, 13.787...|
| 46644840|[[5882422, 3.5798...|
| 57892031|[[5750441, 1.2156...|
| 86002141|[[5809818, 37.532...|
|119056202|[[5688132, 8.6079...|
|129675214|[[5779405, 191.32...|
|131685923|[[5688132, 44.935...|
|137608219|[[5924417, 3.4392...|
|142798053|[[5688132, 14.094...|
|150545114|[[5585638, 15.739...|
|164737612|[[5882422, 10.432...|
|168871476|[[5700029, 74.699...|
|175476266|[[5585632, 16.948...|
|188409888|[[5688132, 85.243...|
|189390340|[[5700032, 110.32...|
|199044224|[[5677934, 13.977...|
|201235176|[[5688132, 7.8814...|
|204142009|[[5700029, 133.32...|
+---------+--------------------+
only showing top 20 rows



In [48]:
userrecs = userRecs.toPandas()

In [49]:
userrecs.head()

,user_id,recommendations
0,40475278,"[(5688132, 21.537269592285156), (5585632, 12.2..."
1,43713532,"[(5585632, 65.2342758178711), (5688132, 61.540..."
2,44772115,"[(5864578, 13.78724479675293), (5842139, 11.05..."
3,46644840,"[(5882422, 3.5798707008361816), (5802431, 2.68..."
4,57892031,"[(5750441, 1.215654969215393), (5820786, 0.999..."


In [50]:
def id_taker(row):
    product_id = ''
    for i in row['recommendations']:
        product_id += str(i['product_id']) + ','
    return product_id

userrecs['new_recommendations'] = userrecs.apply(id_taker, axis=1)
userrecs.head()

,user_id,recommendations,new_recommendations
0,40475278,"[(5688132, 21.537269592285156), (5585632, 12.2...","5688132,5585632,5882421,5760801,5882422,587106..."
1,43713532,"[(5585632, 65.2342758178711), (5688132, 61.540...","5585632,5688132,5700032,5924417,5897532,570004..."
2,44772115,"[(5864578, 13.78724479675293), (5842139, 11.05...","5864578,5842139,5700029,5797129,5822423,570343..."
3,46644840,"[(5882422, 3.5798707008361816), (5802431, 2.68...","5882422,5802431,5814731,5877789,5700049,588242..."
4,57892031,"[(5750441, 1.215654969215393), (5820786, 0.999...","5750441,5820786,5688132,5686925,5877398,586621..."


In [51]:
userrecs = userrecs[['user_id', "new_recommendations"]]
userrecs.columns = ['user_id', 'Recommendations']
userrecs.to_csv('user_product_recomm.csv', index=False)